In [ ]:
import os
import shutil
import pickle
from pathlib import Path


def potsprocess_null_patches(patches_main_folder):

    null_patches_path = os.path.join(patches_main_folder,'Null patches')
    os.makedirs(null_patches_path,exist_ok=True)

    folders_list = ['NIR_SWIR','RGB','VNIR','NIR1']

    for dir in folders_list:
        os.makedirs(os.path.join(null_patches_path,dir),exist_ok=True)

    Path(os.path.join(null_patches_path,'null_events_list.txt')).touch(exist_ok=True)

    already_classified_condition = False

    patches_removed = []

    for notevent_folder in folders_list:
        notevent_folder_path = os.path.join(patches_main_folder,notevent_folder)

        folder_name = notevent_folder

        for notevent_name in os.listdir(notevent_folder_path):
            notevent_path = os.path.join(notevent_folder_path,notevent_name)
            image_name = os.path.basename(notevent_path)
            with open(notevent_path,'rb') as image_file:
                image = pickle.load(image_file)
            
            for j in range(image.shape[2]):
                
                if (image[:,:,j].max(),image[:,:,j].min()) == (0,0):
                    with open(os.path.join(null_patches_path,'null_events_list.txt'),'r') as f:
                        for classified_name in f.readlines():
                            if image_name == classified_name.rstrip('\n'):
                                already_classified_condition =True
                                break
                            else: already_classified_condition = False
                    if not already_classified_condition:
                        with open(os.path.join(null_patches_path,'null_events_list.txt'),'a') as txt_file:
                            txt_file.write(f'{image_name}\n')
                        
                        
                        if image_name.replace(f'_{folder_name}.pkl','') not in patches_removed:
                            for folder in folders_list:
                                folder_id = folder

                                image_folder_name = image_name.replace(folder_name,folder_id)
                                shutil.copyfile(os.path.join(os.path.dirname(notevent_folder_path),folder,image_folder_name),os.path.join(null_patches_path,folder,image_folder_name))
                                
                                os.remove(os.path.join(patches_main_folder,folder,image_folder_name)) # Delete the file from its original location

                            patches_removed.append(image_name.replace(f'_{folder_name}.pkl',''))
    print(f'{len(patches_removed)} null patches have been reclassified. ')

notevents_path = os.path.join(os.getcwd(),'datasets','main_dataset','images','notevent')

potsprocess_null_patches(notevents_path)